In [ ]:
%%writefile README.md
# Geologist_AI

An AI-powered tool that analyzes drill core images to identify rock types using computer vision and machine learning.

##  Features

- **Computer Vision**: Analyzes rock textures and colors using ResNet-18
- **Unsupervised Learning**: Clusters similar rock samples automatically
- **Web Interface**: User-friendly Gradio app for real-time analysis
- **Industry Application**: Solves real geological logging challenges

## How to Use

1. Upload a drill core image
2. Click "Analyze Core Sample"
3. Get instant rock type classification

## Technical Approach

- **Feature Extraction**: ResNet-18 CNN pre-trained on ImageNet
- **Clustering**: K-Means with PCA dimensionality reduction
- **Classification**: Rule-based with color analysis
- **Deployment**: Gradio web interface

## Supported Rock Types

- Gold-bearing rock
- Iron-rich rock
- Lithium-rich rock
- Copper-bearing rock
- Quartz-rich rock
- Waste rock

##  Development
Built with Python, PyTorch, Scikit-learn, and Gradio.

Overwriting README.md


In [ ]:
%%writefile requirements.txt

torch
torchvision
transformers
scikit-learn
Pillow
gradio
requests
numpy
pandas
matplotlib
seaborn

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
%%writefile config.py

import os

# Directories
DATA_DIR = "data"
IMAGE_DIR = os.path.join(DATA_DIR, "core_images")
MODEL_DIR = "models"
OUTPUT_DIR = "output"

# Create directories if they don't exist
os.makedirs(IMAGE_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Model parameters - adaptive to data size
NUM_CLUSTERS = 3  # Reduced default
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Candidate labels for classification
CANDIDATE_LABELS = [
    "gold-bearing rock",
    "iron-rich rock",
    "lithium-rich rock",
    "copper-bearing rock",
    "waste rock",
    "quartz-rich rock",
    "sulfide-rich rock"
]

# Public geology repositories
DATASET_SOURCES = [
    {
        "name": "Geoscience Australia",
        "url": "https://geology.csiro.au/datasets/drill-core-images",
        "description": "Australian geological survey drill core images"
    },
    {
        "name": "USGS Mineral Resources",
        "url": "https://mrdata.usgs.gov/geology/state/map-viewer.php",
        "description": "US Geological Survey mineral resources data"
    },
    {
        "name": "BGS OpenGeoscience",
        "url": "https://www.bgs.ac.uk/discovering-geology/rock-library/",
        "description": "British Geological Survey rock sample images"
    }
]

Writing config.py


In [ ]:
%%writefile data_collector.py

import os
import requests
from PIL import Image
from io import BytesIO
import time
from config import IMAGE_DIR, DATASET_SOURCES

class DataCollector:
    def __init__(self):
        self.image_dir = IMAGE_DIR
        self.sources = DATASET_SOURCES

    def collect_sample_images(self):
        """Collect sample images from public sources"""
        # These are example URLs - in practice you'd scrape or use APIs
        sample_urls = [
            "https://c7.alamy.com/comp/3AJ86J0/gold-on-quartz-bradshaw-mountains-arizona-gold-on-quartz-from-the-bradshaw-mountains-arizona-is-a-classic-and-highly-sought-after-mineral-associa-3AJ86J0.jpg",
            "https://www.nuggetsbygrant.com/cdn/shop/products/243A0948.jpg?v=1670014792&width=1080",
            "https://news.rice.edu/sites/g/files/bxs2656/files/inline-images/BIF5-0524_540_1.jpeg",
            "https://c7.alamy.com/comp/2FNKTF3/copper-bearing-rock-against-a-gravel-ground-surface-2FNKTF3.jpg",
            "https://www.shutterstock.com/shutterstock/photos/2618131965/display_1500/stock-photo-close-up-of-a-rough-weathered-copper-ore-stone-with-natural-crystal-formations-2618131965.jpg",
            "https://geologyistheway.com/wp-content/uploads/2021/06/118-milky-quartz.jpg",
            "https://geologyistheway.com/wp-content/uploads/2021/06/201210-4-1024x726.jpg"


        ]



        print("Collecting sample drill core images...")
        for i, url in enumerate(sample_urls):
            try:
                response = requests.get(url, timeout=10)
                response.raise_for_status()

                img = Image.open(BytesIO(response.content))
                img_path = os.path.join(self.image_dir, f"sample_core_{i+1}.jpg")
                img.save(img_path)
                print(f"Downloaded: sample_core_{i+1}.jpg")
                time.sleep(0.5)  # Be respectful to servers
            except Exception as e:
                print(f"Failed to download {url}: {e}")

        print(f"Collected {len(os.listdir(self.image_dir))} images")

    def get_dataset_info(self):
        """Return information about available datasets"""
        return self.sources

if __name__ == "__main__":
    collector = DataCollector()
    collector.collect_sample_images()
    print("\nAvailable geological datasets:")
    for source in collector.get_dataset_info():
        print(f"- {source['name']}: {source['description']}")
        print(f"  URL: {source['url']}\n")

Writing data_collector.py


In [ ]:
%%writefile core_dataset.py

import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from config import IMAGE_SIZE

class CoreDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_paths = [
            os.path.join(image_dir, f)
            for f in os.listdir(image_dir)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        self.transform = transform or self.default_transform()

    def default_transform(self):
        return transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path

Writing core_dataset.py


In [ ]:
%%writefile feature_extractor.py

import torch
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
import numpy as np
from core_dataset import CoreDataset
from config import BATCH_SIZE

class FeatureExtractor:
    def __init__(self, device=None):
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._load_model()

    def _load_model(self):
        """Load pretrained ResNet18 and remove classification layer"""
        weights = ResNet18_Weights.DEFAULT
        model = resnet18(weights=weights)
        # Remove the final classification layer
        model = nn.Sequential(*list(model.children())[:-1])
        model = model.to(self.device)
        model.eval()
        return model

    def extract_features(self, image_dir):
        """Extract features from all images in directory"""
        dataset = CoreDataset(image_dir)
        dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

        features = []
        image_paths = []

        print("Extracting features from images...")
        with torch.no_grad():
            for batch, paths in dataloader:
                batch = batch.to(self.device)
                batch_features = self.model(batch)
                batch_features = batch_features.view(batch_features.size(0), -1)
                features.append(batch_features.cpu().numpy())
                image_paths.extend(paths)

        features = np.vstack(features)
        print(f"Extracted features shape: {features.shape}")
        return features, image_paths

if __name__ == "__main__":
    from config import IMAGE_DIR
    extractor = FeatureExtractor()
    features, paths = extractor.extract_features(IMAGE_DIR)
    print(f"Extracted features for {len(paths)} images")

Writing feature_extractor.py


In [ ]:
%%writefile cluster_analyzer.py

import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from config import NUM_CLUSTERS, OUTPUT_DIR
import os

class ClusterAnalyzer:
    def __init__(self, n_clusters=NUM_CLUSTERS):
        self.n_clusters = n_clusters
        self.scaler = StandardScaler()
        self.kmeans = None
        self.pca = None

    def fit_predict(self, features):
        """Fit KMeans and return cluster labels"""
        # Standardize features
        features_scaled = self.scaler.fit_transform(features)

        # Adaptive PCA
        n_components = min(features_scaled.shape[0] - 1, features_scaled.shape[1], 50)
        if n_components < 1:
            n_components = 1

        print(f"Using {n_components} PCA components (adapted to data size)")
        self.pca = PCA(n_components=n_components)
        features_reduced = self.pca.fit_transform(features_scaled)

        # Adjust number of clusters if needed
        n_clusters = min(self.n_clusters, len(features_reduced))
        if n_clusters < 1:
            n_clusters = 1

        print(f"Using {n_clusters} clusters")
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = self.kmeans.fit_predict(features_reduced)
        return labels, features_reduced

    def get_cluster_centers(self):
        """Return cluster centers"""
        if self.kmeans is not None:
            return self.kmeans.cluster_centers_
        return None

    def visualize_clusters(self, features, labels, image_paths, save_path=None):
        """Visualize clusters using PCA"""

        if features.shape[0] > 2 and features.shape[1] > 2:
            pca_2d = PCA(n_components=min(2, features.shape[0] - 1, features.shape[1]))
            features_2d = pca_2d.fit_transform(features)
        else:

            features_2d = features[:, :2] if features.shape[1] >= 2 else np.hstack([features, np.zeros((features.shape[0], 2 - features.shape[1]))])

        # Create plot
        plt.figure(figsize=(12, 8))

        # Handle case where we have only one cluster
        unique_labels = np.unique(labels)
        if len(unique_labels) > 1:
            scatter = plt.scatter(features_2d[:, 0], features_2d[:, 1], c=labels, cmap='tab10', alpha=0.7, s=100)
            plt.colorbar(scatter)
        else:
            plt.scatter(features_2d[:, 0], features_2d[:, 1], c='blue', alpha=0.7, s=100)
            plt.title(f'All samples in single cluster (Cluster {labels[0]})')

        plt.title('Drill Core Sample Clusters (PCA Visualization)', fontsize=16)
        plt.xlabel('Feature Dimension 1')
        plt.ylabel('Feature Dimension 2')

        # Annotate some points
        for i in range(min(15, len(features_2d))):
            if i < len(image_paths):
                filename = os.path.basename(image_paths[i])[:15] + "..."
                plt.annotate(filename, (features_2d[i, 0], features_2d[i, 1]),
                            xytext=(5, 5), textcoords='offset points', fontsize=8, alpha=0.7)

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"Cluster visualization saved to {save_path}")
        plt.show()

    def create_cluster_map(self, image_paths, labels):
        """Create mapping from cluster ID to image paths"""
        cluster_map = {}
        for path, label in zip(image_paths, labels):
            if label not in cluster_map:
                cluster_map[label] = []
            cluster_map[label].append(path)
        return cluster_map

    def analyze_cluster_characteristics(self, features, labels, image_paths):
        """Analyze characteristics of each cluster"""
        cluster_stats = {}

        # Get features for each cluster
        for cluster_id in np.unique(labels):
            mask = labels == cluster_id
            cluster_features = features[mask]

            # Calculate statistics
            mean_features = np.mean(cluster_features, axis=0)
            std_features = np.std(cluster_features, axis=0)

            # Get image paths for this cluster
            cluster_images = [path for i, path in enumerate(image_paths) if labels[i] == cluster_id]

            cluster_stats[cluster_id] = {
                'count': len(cluster_images),
                'mean_features': mean_features,
                'std_features': std_features,
                'sample_images': cluster_images[:5]  # First 5 samples
            }

        return cluster_stats

    def analyze_clusters(self, features, image_paths):
        """Complete clustering analysis"""
        print(f"Performing clustering analysis on {len(image_paths)} samples...")
        print(f"Feature dimensions: {features.shape}")

        # Perform clustering
        labels, features_reduced = self.fit_predict(features)

        # Create cluster map
        cluster_map = self.create_cluster_map(image_paths, labels)

        # Analyze cluster characteristics
        cluster_stats = self.analyze_cluster_characteristics(features, labels, image_paths)

        # Visualize if we have enough samples
        if len(image_paths) > 2:
            viz_path = os.path.join(OUTPUT_DIR, "clusters.png")
            self.visualize_clusters(features, labels, image_paths, viz_path)

        # Print cluster information
        print("\n" + "="*60)
        print("CLUSTER ANALYSIS RESULTS")
        print("="*60)
        for cluster_id, stats in cluster_stats.items():
            print(f"\nCluster {cluster_id}:")
            print(f"  Samples: {stats['count']} images")
            print(f"  Sample files:")
            for path in stats['sample_images']:
                print(f"    - {os.path.basename(path)}")

        return labels, cluster_map, cluster_stats

if __name__ == "__main__":
    pass

Writing cluster_analyzer.py


In [ ]:
%%writefile gen_ai_labeler.py

from transformers import pipeline
import torch
import os
from PIL import Image
import torchvision.transforms as transforms
from config import CANDIDATE_LABELS, IMAGE_SIZE

class GenAILabeler:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.classifier = pipeline(
            "zero-shot-classification",
            model="facebook/bart-large-mnli",
            device=0 if torch.cuda.is_available() else -1
        )
        # More specific candidate labels
        self.candidate_labels = CANDIDATE_LABELS

    def analyze_image_content(self, image_path):
        """Extract visual characteristics from image filename"""

        # For now, we'll create better prompts based on filenamr
        filename = os.path.basename(image_path).lower()

        characteristics = []
        if 'gold' in filename:
            characteristics.append("visible metallic particles, yellow coloration")
        if 'iron' in filename or 'pyrite' in filename:
            characteristics.append("dark metallic appearance, magnetic properties")
        if 'lithium' in filename or 'spodumene' in filename:
            characteristics.append("light-colored minerals, pegmatite texture")
        if 'copper' in filename:
            characteristics.append("green or blue coloration, metallic luster")
        if 'quartz' in filename:
            characteristics.append("clear or white crystalline structure")
        if 'granite' in filename:
            characteristics.append("mixed mineral composition, coarse-grained")
        if 'basalt' in filename:
            characteristics.append("dark fine-grained texture")

        if not characteristics:
            characteristics = ["visible mineral grains", "distinctive color patterns", "unique textural features"]

        return ", ".join(characteristics)

    def label_cluster(self, sample_image_path):
        """Generate label for a cluster based on a sample image"""
        # Get visual characteristics
        visual_features = self.analyze_image_content(sample_image_path)

        # Create a more specific prompt
        prompt = f"A geological drill core sample showing {visual_features}. "
        prompt += "What economically important mineral is most likely present in this rock sample?"

        # Perform zero-shot classification
        result = self.classifier(prompt, self.candidate_labels)

        # Return top prediction with all scores
        return {
            "label": result['labels'][0],
            "confidence": result['scores'][0],
            "all_scores": dict(zip(result['labels'], result['scores'])),
            "prompt_used": prompt
        }

    def label_all_clusters(self, cluster_map):
        """Label all clusters with improved context"""
        cluster_labels = {}

        print("Generating detailed labels for clusters using GenAI...")
        for cluster_id, image_paths in cluster_map.items():
            # Use first image as sample for the cluster
            sample_path = image_paths[0]
            label_info = self.label_cluster(sample_path)
            cluster_labels[cluster_id] = label_info

            print(f"\nCluster {cluster_id}:")
            print(f"  Primary Label: {label_info['label']}")
            print(f"  Confidence: {label_info['confidence']:.3f}")
            print(f"  Key Features: {self.analyze_image_content(sample_path)}")

            # Show top 3 alternative labels
            sorted_scores = sorted(label_info['all_scores'].items(), key=lambda x: x[1], reverse=True)
            print("  Alternative possibilities:")
            for label, score in sorted_scores[1:4]:
                print(f"    - {label}: {score:.3f}")

        return cluster_labels

if __name__ == "__main__":

    pass

Writing gen_ai_labeler.py


In [ ]:
%%writefile simple_classifier.py


import os
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from config import CANDIDATE_LABELS
import torch
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn

class SimpleRockClassifier:
    def __init__(self):
        # Load pre-trained model
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

        # Load ResNet model
        weights = ResNet18_Weights.DEFAULT
        self.model = resnet18(weights=weights)
        self.model = nn.Sequential(*list(self.model.children())[:-1])  # Remove final layer
        self.model.eval()


        self.keyword_mapping = {
            'gold': 'gold-bearing rock',
            'iron': 'iron-rich rock',
            'pyrite': 'iron-rich rock',
            'lithium': 'lithium-rich rock',
            'spodumene': 'lithium-rich rock',
            'copper': 'copper-bearing rock',
            'quartz': 'quartz-rich rock',
            'silica': 'quartz-rich rock',
            'crystal': 'quartz-rich rock',
            'waste': 'waste rock',
            'granite': 'waste rock',
            'basalt': 'waste rock'
        }

    def extract_features(self, image_path):
        """Extract features from image"""
        try:
            image = Image.open(image_path).convert("RGB")
            image_tensor = self.transform(image).unsqueeze(0)

            with torch.no_grad():
                features = self.model(image_tensor)
                features = features.view(features.size(0), -1)

            return features.numpy()
        except Exception as e:
            print(f"Error extracting features: {e}")
            return np.random.rand(1, 512)

    def classify_by_filename(self, image_path):
        """Classify based on filename keywords"""
        filename = os.path.basename(image_path).lower()

        for keyword, rock_type in self.keyword_mapping.items():
            if keyword in filename:
                return rock_type, 0.8


        return self.analyze_colors(image_path)

    def analyze_colors(self, image_path):
        """Simple color analysis"""
        try:
            image = Image.open(image_path).convert("RGB")
            # Resize for faster processing
            image_small = image.resize((50, 50))
            pixels = np.array(image_small)

            # Calculate average color
            mean_color = np.mean(pixels, axis=(0, 1))

            # Simple color-based classification
            r, g, b = mean_color

            # Gold detection (yellow)
            if r > 180 and g > 150 and b < 100 and r > g > b:
                return "gold-bearing rock", 0.7

            # Iron detection (dark)
            if (r + g + b) / 3 < 100:
                return "iron-rich rock", 0.65

            # Copper detection (green/blue)
            if g > r and g > b and (r + g + b) / 3 > 80:
                return "copper-bearing rock", 0.6

            # Light minerals (lithium/quartz)
            if (r + g + b) / 3 > 200:
                # Check for purple tint (lithium)
                if abs(r - b) < 30 and (r + g + b) / 3 > 220:
                    return "lithium-rich rock", 0.55
                else:
                    return "quartz-rich rock", 0.7

            return "waste rock", 0.5

        except Exception as e:
            print(f"Error in color analysis: {e}")
            return "waste rock", 0.3

    def predict(self, image_path):
        """Main prediction function"""
        # First try filename-based classification
        rock_type, confidence = self.classify_by_filename(image_path)

        # Extract features for potential future use
        features = self.extract_features(image_path)

        return {
            "rock_type": rock_type,
            "confidence": confidence,
            "features": features,
            "explanation": f"Classified as {rock_type} based on visual characteristics"
        }

Overwriting simple_classifier.py


In [ ]:
%%writefile model.py

import os
import pickle
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights
import torchvision.transforms as transforms
from PIL import Image
from feature_extractor import FeatureExtractor
from cluster_analyzer import ClusterAnalyzer
from rock_classifier import RockClassifier
from config import MODEL_DIR
from sklearn.metrics.pairwise import cosine_distances

class CoreLoggerModel:
    def __init__(self):
        self.feature_extractor = FeatureExtractor()
        self.cluster_analyzer = ClusterAnalyzer()
        self.rock_classifier = RockClassifier()

        self.features = None
        self.image_paths = None
        self.cluster_labels = None
        self.cluster_map = None
        self.cluster_stats = None
        self.trained = False
        self.reduced_cluster_centers = None  #s
        self.pca_transformer = None

        # For prediction
        self.prediction_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

    def extract_features_for_prediction(self, image_path):
        """Extract features from a single image for prediction"""
        image = Image.open(image_path).convert("RGB")
        image_tensor = self.prediction_transform(image).unsqueeze(0)

        with torch.no_grad():
            features = self.feature_extractor.model(image_tensor)
            features = features.view(features.size(0), -1)

        return features.numpy()

    def train(self, image_dir):
        """Train the model on drill core images"""
        print("Training AI Geologist model...")
        print("Step 1: Extracting visual features...")

        # Step 1: Extract features
        self.features, self.image_paths = self.feature_extractor.extract_features(image_dir)

        print("Step 2: Performing unsupervised clustering...")
        # Step 2: Perform clustering
        labels, self.cluster_map, self.cluster_stats = self.cluster_analyzer.analyze_clusters(
            self.features, self.image_paths
        )

        # Store PCA transformer and reduced cluster centers for prediction
        self.pca_transformer = self.cluster_analyzer.pca
        self.reduced_cluster_centers = self.cluster_analyzer.kmeans.cluster_centers_

        print("Step 3: Classifying clusters based on visual features...")
        # Step 3: Classify clusters using rule-based approach
        self.cluster_labels = self.rock_classifier.classify_all_clusters(self.cluster_stats)

        self.trained = True
        print("\n Model training completed successfully!")
        return self

    def predict(self, image_path):
        """Actually predict cluster for a new image"""
        if not self.trained:
            raise ValueError("Model must be trained before making predictions")

        if self.reduced_cluster_centers is None or self.pca_transformer is None:
            raise ValueError("Model not properly trained - missing cluster centers")

        # Extract features from the uploaded image
        new_features = self.extract_features_for_prediction(image_path)

        # Standardize features using the same scaler from training
        new_features_scaled = self.cluster_analyzer.scaler.transform(new_features)

        # Apply PCA transformation using the same transformer from training
        new_features_reduced = self.pca_transformer.transform(new_features_scaled)

        # Find the closest cluster center
        distances = cosine_distances(new_features_reduced, self.reduced_cluster_centers)
        predicted_cluster = np.argmin(distances)

        # Get the label for this cluster
        if predicted_cluster in self.cluster_labels:
            label_info = self.cluster_labels[predicted_cluster]
        else:

            cluster_ids = list(self.cluster_labels.keys())
            fallback_cluster = cluster_ids[0] if cluster_ids else 0
            label_info = self.cluster_labels.get(fallback_cluster, {
                'label': 'unknown rock',
                'confidence': 0.5,
                'all_scores': {},
                'dominant_color': [128, 128, 128],
                'brightness': 128
            })


        response = {
            "predicted_cluster": int(predicted_cluster),
            "rock_type": label_info["label"],
            "confidence": float(label_info["confidence"]),
            "explanation": f"This drill core sample has been classified as {label_info['label']} "
                          f"based on its visual characteristics including color (RGB: "
                          f"{[int(x) for x in label_info['dominant_color']]}) and brightness "
                          f"({label_info['brightness']:.1f}).",
            "alternative_possibilities": []
        }

        # Add alternative possibilities
        sorted_scores = sorted(label_info['all_scores'].items(), key=lambda x: x[1], reverse=True)
        for label, score in sorted_scores[1:4]:  # Top 3 alternatives
            if score > 0.05:  # Only show if score is reasonable
                response["alternative_possibilities"].append({
                    "rock_type": label,
                    "confidence": float(score)
                })

        return response

    def get_model_summary(self):
        """Get a summary of the trained model"""
        if not self.trained:
            return "Model not trained yet"

        summary = f"AI Geologist Model Summary:\n"
        summary += f"- Trained on {len(self.image_paths)} drill core images\n"
        summary += f"- Identified {len(self.cluster_labels)} distinct visual clusters\n\n"

        for cluster_id, label_info in self.cluster_labels.items():
            summary += f"Cluster {cluster_id}: {label_info['label']} "
            summary += f"(confidence: {label_info['confidence']:.2f})\n"

        return summary

    def save_model(self, filepath=None):
        """Save the trained model"""
        if filepath is None:
            filepath = os.path.join(MODEL_DIR, "core_logger_model.pkl")

        model_data = {
            'features': self.features,
            'image_paths': self.image_paths,
            'cluster_labels': self.cluster_labels,
            'cluster_map': self.cluster_map,
            'cluster_stats': self.cluster_stats,
            'reduced_cluster_centers': self.reduced_cluster_centers,
            'trained': self.trained,
            'pca_transformer': self.pca_transformer,
            'scaler': self.cluster_analyzer.scaler
        }

        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)

        print(f"Model saved to {filepath}")

    def load_model(self, filepath=None):
        """Load a trained model"""
        if filepath is None:
            filepath = os.path.join(MODEL_DIR, "core_logger_model.pkl")

        if not os.path.exists(filepath):
            print(f"Model file not found: {filepath}")
            return self

        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)

        self.features = model_data['features']
        self.image_paths = model_data['image_paths']
        self.cluster_labels = model_data['cluster_labels']
        self.cluster_map = model_data['cluster_map']
        self.cluster_stats = model_data['cluster_stats']
        self.reduced_cluster_centers = model_data['reduced_cluster_centers']
        self.trained = model_data.get('trained', False)

        # Restore PCA transformer and scaler
        if 'pca_transformer' in model_data:
            self.pca_transformer = model_data['pca_transformer']
        if 'scaler' in model_data:
            self.cluster_analyzer.scaler = model_data['scaler']

        print(f"Model loaded from {filepath}")
        return self

if __name__ == "__main__":
    from config import IMAGE_DIR
    model = CoreLoggerModel()
    model.train(IMAGE_DIR)
    model.save_model()

    # Print model summary
    print("\n" + "="*50)
    print(model.get_model_summary())
    print("="*50)

Overwriting model.py


In [ ]:
%%writefile app.py


import gradio as gr
import os
from simple_classifier import SimpleRockClassifier

# Initialize classifier
classifier = SimpleRockClassifier()

def analyze_core(image):
    """Analyze a drill core image"""
    # Save uploaded image temporarily
    temp_path = "temp_upload.jpg"
    image.save(temp_path)

    # Get prediction
    try:
        result = classifier.predict(temp_path)


        response = f"""
        ##  Drill Core Analysis Results

        ### Primary Prediction
        **Rock Type:** `{result['rock_type']}`
        **Confidence:** `{result['confidence']:.2f}`

        ### Analysis Details
        {result['explanation']}
        """

    except Exception as e:
        response = f"##  Error\nAn error occurred during analysis: {str(e)}"

    # Clean
    if os.path.exists(temp_path):
        os.remove(temp_path)

    return response

# Create Gradio interface
with gr.Blocks(title="Geologist_AI - Core Logger") as demo:
    gr.Markdown("#  Geologist_AI - Core Logger")
    gr.Markdown("Upload a drill core image to identify the rock type")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="📷 Drill Core Image")
            submit_btn = gr.Button("🔍 Analyze Core Sample", variant="primary")
        with gr.Column():
            output_text = gr.Markdown(label="📊 Analysis Results")

    submit_btn.click(
        fn=analyze_core,
        inputs=image_input,
        outputs=output_text
    )

    gr.Markdown("---")
    gr.Markdown("### About this Tool")
    gr.Markdown("""
    This AI-powered geologist identifies rock types based on:
    - **Visual color analysis**
    - **Deep learning feature extraction**

    **Supported rock types:**
    - Gold-bearing rock
    - Iron-rich rock
    - Lithium-rich rock
    - Copper-bearing rock
    - Quartz-rich rock
    - Waste rock
    """)

# Launch
if __name__ == "__main__":
    demo.launch()

Overwriting app.py


In [ ]:
!python data_collector.py

Downloaded: sample_core_1.jpg
Downloaded: sample_core_2.jpg
Downloaded: sample_core_3.jpg
Downloaded: sample_core_4.jpg
Downloaded: sample_core_5.jpg
Failed to download https://geologyistheway.com/wp-content/uploads/2021/06/118-milky-quartz.jpg: 403 Client Error: Forbidden for url: https://geologyistheway.com/wp-content/uploads/2021/06/118-milky-quartz.jpg
Failed to download https://geologyistheway.com/wp-content/uploads/2021/06/201210-4-1024x726.jpg: 403 Client Error: Forbidden for url: https://geologyistheway.com/wp-content/uploads/2021/06/201210-4-1024x726.jpg
Collected 5 images

Available geological datasets:
- Geoscience Australia: Australian geological survey drill core images
  URL: https://geology.csiro.au/datasets/drill-core-images

- USGS Mineral Resources: US Geological Survey mineral resources data
  URL: https://mrdata.usgs.gov/geology/state/map-viewer.php

- BGS OpenGeoscience: British Geological Survey rock sample images
  URL: https://www.bgs.ac.uk/discovering-geology/r

In [ ]:
!python model.py

Training AI Geologist model...
Step 1: Extracting visual features...
Extracting features from images...
Extracted features shape: (5, 512)
Step 2: Performing unsupervised clustering...
Performing clustering analysis on 5 samples...
Feature dimensions: (5, 512)
Using 4 PCA components (adapted to data size)
Using 3 clusters
Cluster visualization saved to output/clusters.png
Figure(1200x800)

CLUSTER ANALYSIS RESULTS

Cluster 0:
  Samples: 3 images
  Sample files:
    - sample_core_1.jpg
    - sample_core_2.jpg
    - sample_core_5.jpg

Cluster 1:
  Samples: 1 images
  Sample files:
    - sample_core_4.jpg

Cluster 2:
  Samples: 1 images
  Sample files:
    - sample_core_3.jpg
Step 3: Classifying clusters based on visual features...

CLUSTER CLASSIFICATION RESULTS

Cluster 0:
  Predicted: quartz-rich rock
  Confidence: 0.906
  Dominant color: RGB(183, 175, 166)
  Brightness: 175.2

Cluster 1:
  Predicted: waste rock
  Confidence: 1.000
  Dominant color: RGB(145, 107, 86)
  Brightness: 113.

In [ ]:
!python app.py

Model loaded from models/core_logger_model.pkl
Loaded existing model
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://24b284703479000d24.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 3107, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/app.py", line 103, in <module>
    demo.launch(share=True)
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 3013, in launch
    self.block_thread()
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 3111, in block_thread
 

In [ ]:
!ls -la

total 32
drwxr-xr-x 3 root root 4096 Jul 29 18:23 .
drwxr-xr-x 5 root root 4096 Jul 29 18:06 ..
-rw-r--r-- 1 root root 2050 Jul 29 18:35 app.py
drwxr-xr-x 8 root root 4096 Jul 29 18:06 .git
-rw-r--r-- 1 root root 1519 Jul 29 18:06 .gitattributes
-rw-r--r-- 1 root root  326 Jul 29 18:06 README.md
-rw-r--r-- 1 root root 4417 Jul 29 18:35 simple_classifier.py
